In [1]:
from kaiseki import lattice
from kaiseki import kakunou_kai
from kaiseki import Short_Algo_kai
from dir_sci import Short_sci
from dir_exp_sci import exp_Short_sci
from nume_ex_prm import Short_FISTA
from exp_nume_ex import exp_Short_FISTA
from nume_ex_prm import Short_qNewton
import time
import numpy as np
import pandas as pd
import scipy.optimize as optimize
import scipy.sparse as spsp
import matplotlib.pyplot as plt
import csv
import os

from scipy.spatial import distance
from scipy.optimize import linprog
from scipy.optimize import minimize
from collections import defaultdict
from mpl_toolkits.mplot3d import Axes3D

## 解析解を求める

In [ ]:
E_list = [2, 5, 8, 10]
Col_list = [20, 30]

# Scaling * tが格子の最小距離となる。
Scaling = 3 / Col

K = Col * Col

M = 0.1 * K
N = 0.1 * K

dF_long = pd.read_excel(fr'C:/Users/kimura/OneDrive - 国立大学法人東北大学/numerical/short_dual/log/excelfile/' \
    fr'shortlong/kaisekikai/theta=5.0/mesh/E={E}/v_proj=0.1/K={Col}^2/truevalue/jikken_F.xlsx')
dH_long = pd.read_excel(fr'C:/Users/kimura/OneDrive - 国立大学法人東北大学/numerical/short_dual/log/excelfile/' \
    fr'shortlong/kaisekikai/theta=5.0/mesh/E={E}/v_proj=0.1/K={Col}^2/truevalue/jikken_H.xlsx')

m_fixed = np.array(dF_long['m'].tolist())
n_fixed = np.array(dH_long['n'].tolist()).reshape(K, K)

# m_per = M / K
# m_fixed = np.array([m_per] * K)
# n_fixed = np.full((K, K), N / (K * K))

city_network = lattice_kai.make_lattice(Col)

# 通勤費用パラメータ
t = 0.1

# 距離抵抗パラメータ
tau = 0.01

# 総土地供給量
S_total = K
S_bar = S_total / K

Coordinate_Data = \
np.array([(city_network['node_dic'][str(i)]['x_pos']*Scaling,
           city_network['node_dic'][str(i)]['y_pos']*Scaling) for i in range(K)])
distance_matrix = distance.squareform(distance.pdist(Coordinate_Data))

theta_N = 5.0
theta_M = 5.0

# E = 5

alpha_1 = 0.4
alpha_2 = 0.4
beta_1 = 0.4
beta_2 = 0.4

#パラメータ設定
prm = Short_Algo_kai.Parameter(Col, distance_matrix, t, tau, Scaling, S_total, S_bar,
            theta_M, theta_N, E, alpha_1, alpha_2, beta_1, beta_2, M, N)

R_true = ((prm.alpha_1 * np.sum(n_fixed, axis=1) + prm.beta_1 * m_fixed) / prm.S_bar)
W_true = (1 / prm.E) * (prm.beta_2 * (m_fixed / np.sum(n_fixed, axis=0)) + prm.alpha_2)
v_true = prm.E * W_true * np.ones((prm.K, prm.K))\
        - (prm.alpha_1 * np.log(R_true) * np.ones((prm.K, prm.K))).T\
        - prm.alpha_2 * np.log(W_true) * np.ones((prm.K, prm.K))\
        + prm.alpha_1 * np.log(prm.alpha_1) + prm.alpha_2 * np.log(prm.alpha_2)\
        - prm.alpha_1 - prm.alpha_2 - prm.T
pi_true = np.dot(prm.D, m_fixed)\
        - prm.beta_1 * np.log(R_true) - prm.beta_2 * np.log(W_true)\
        + prm.beta_1 * np.log(prm.beta_1) + prm.beta_2 * np.log(prm.beta_2)\
        - prm.beta_1 - prm.beta_2

In [6]:
E_list = [2, 5, 8, 10]
Col_list = [5, 8, 10]
for Col in Col_list:
    
    for E in E_list:
        
        print("="*40)
        print(f'E={E}, K={Col}^2')
        
        K = Col * Col
    
        M = 0.1 * K
        N = 0.1 * K

        dF_long = pd.read_excel(fr'C:/Users/kimura/OneDrive - 国立大学法人東北大学/numerical/short_dual/log/excelfile/' \
                fr'shortlong/kaisekikai/theta=5.0/mesh/E={E}/v_proj=0.1/K={Col}^2/truevalue/jikken_F.xlsx')
        dH_long = pd.read_excel(fr'C:/Users/kimura/OneDrive - 国立大学法人東北大学/numerical/short_dual/log/excelfile/' \
                fr'shortlong/kaisekikai/theta=5.0/mesh/E={E}/v_proj=0.1/K={Col}^2/truevalue/jikken_H.xlsx')

        m_fixed = np.array(dF_long['m'].tolist())
        n_fixed = np.array(dH_long['n'].tolist()).reshape(K, K)

        # m_per = M / K
        # m_fixed = np.array([m_per] * K)
        # n_fixed = np.full((K, K), N / (K * K))
        
        Scaling = 3 / Col

        city_network = lattice_kai.make_lattice(Col)

        # 通勤費用パラメータ
        t = 0.1
        
        # 距離抵抗パラメータ
        tau = 0.01

        # 総土地供給量
        S_total = K
        S_bar = S_total / K

        Coordinate_Data = \
        np.array([(city_network['node_dic'][str(i)]['x_pos']*Scaling,
                   city_network['node_dic'][str(i)]['y_pos']*Scaling) for i in range(K)])
        distance_matrix = distance.squareform(distance.pdist(Coordinate_Data))

        theta_N = 5.0
        theta_M = 5.0

        # E = 5

        alpha_1 = 0.4
        alpha_2 = 0.4
        beta_1 = 0.4
        beta_2 = 0.4

        L = 0.2
        eta = 1.5
        p_proj = 1e-3

        #パラメータ設定
        prm = Short_FISTA.Parameter(Col, distance_matrix, t, tau, Scaling, S_total, S_bar,
                    theta_M, theta_N, E, alpha_1, alpha_2, beta_1, beta_2, M, N)
        algprm = Short_FISTA.Algo_Parameter(L, eta, p_proj)

        dF_FISTA = pd.read_excel(fr'C:/Users/kimura/OneDrive - 国立大学法人東北大学/numerical/short_dual/log/excelfile/' \
            fr'short/FISTA/long_shuseki/mesh/E={prm.E}/L={algprm.L}/p_proj={algprm.p_proj}/eta={algprm.eta}/v_proj=None/df=None/K={prm.Col}^2/truevalue/jikken_F.xlsx')
        dF_kaiseki = pd.read_excel(fr'C:/Users/kimura/OneDrive - 国立大学法人東北大学/numerical/short_dual/log/excelfile/' \
            fr'short/kaisekikai/long_shuseki/mesh/E={prm.E}/v_proj=None/K={prm.Col}^2/jikken_F.xlsx')
        dH_FISTA = pd.read_excel(fr'C:/Users/kimura/OneDrive - 国立大学法人東北大学/numerical/short_dual/log/excelfile/' \
            fr'short/FISTA/long_shuseki/mesh/E={prm.E}/L={algprm.L}/p_proj={algprm.p_proj}/eta={algprm.eta}/v_proj=None/df=None/K={prm.Col}^2/truevalue/jikken_H.xlsx')
        dH_kaiseki = pd.read_excel(fr'C:/Users/kimura/OneDrive - 国立大学法人東北大学/numerical/short_dual/log/excelfile/' \
            fr'short/kaisekikai/long_shuseki/mesh/E={prm.E}/v_proj=None/K={prm.Col}^2/jikken_H.xlsx')

        R_FISTA = np.array(dF_FISTA['R'].tolist())
        R_kaiseki = np.array(dF_kaiseki['R'].tolist())

        print("R_rel_err:", np.mean(abs((R_FISTA - R_kaiseki) / R_kaiseki)))
        # print("R_FISTA:", np.mean(R_FISTA))
        # print("R_kaiseki:", np.mean(R_kaiseki))

        W_FISTA = np.array(dF_FISTA['W'].tolist())
        W_kaiseki = np.array(dF_kaiseki['W'].tolist())

        print("W_rel_err:", np.mean(abs((W_FISTA - W_kaiseki) / W_kaiseki)))
        # print("W_FISTA:", np.mean(W_FISTA))
        # print("W_kaiseki:", np.mean(W_kaiseki))

        pi_FISTA = np.array(dF_FISTA['pi'].tolist())
        pi_kaiseki = np.array(dF_kaiseki['pi'].tolist())

        print("pi_rel_err:", np.mean(abs((pi_FISTA - pi_kaiseki) / pi_kaiseki)))
        # print("pi_FISTA:", np.mean(pi_FISTA))
        # print("pi_kaiseki:", np.mean(pi_kaiseki))

        v_FISTA = np.array(dH_FISTA['v'].tolist()).flatten()
        v_kaiseki = np.array(dH_kaiseki['v'].tolist()).flatten()

        v_err = []
        for i in range(len(v_FISTA)):
            if v_FISTA[i] == 0:  # ゼロ割りを防ぐための処理
                if v_kaiseki[i] == 0:
                    v_err.append(0)
                else:
                    v_err.append(float('inf'))
            else:
                v_err.append(abs((v_FISTA[i] - v_kaiseki[i]) / v_kaiseki[i]))
            
        # print("v_err:", v_err)
        
        print("v_rel_err:", np.mean(v_err))
        # print("v_FISTA:", v_FISTA)
        # print("v_kaiseki:", v_kaiseki)
        # print("v_FISTA_min:", np.min(v_FISTA))
        # print("v_kaiseki_min:", np.min(v_kaiseki))
        
        short = Short_FISTA.Short(prm, algprm, m_fixed, n_fixed)
        RW_FISTA = np.concatenate((R_FISTA, W_FISTA))
        dRdW_FISTA = short.short_dual_df(RW_FISTA)
        
        dR_FISTA = dRdW_FISTA[:K]
        dW_FISTA = dRdW_FISTA[K:]
        
        print("dR_FISTA:", np.mean(dR_FISTA))
        print("dW_FISTA:", np.mean(dW_FISTA))
        
        short = Short_FISTA.Short(prm, algprm, m_fixed, n_fixed)
        RW_kaiseki = np.concatenate((R_kaiseki, W_kaiseki))
        dRdW_kaiseki = short.short_dual_df(RW_kaiseki)
        
        dR_kaiseki = dRdW_kaiseki[:K]
        dW_kaiseki = dRdW_kaiseki[K:]
        
        print("dR_kaiseki:", np.mean(dR_kaiseki))
        print("dW_kaiseki:", np.mean(dW_kaiseki))

        #objはデータに格納されていない．
        # obj_sci = np.array(dF_sci['v'].tolist())
        # obj_FISA = np.array(dF_FISTA['v'].tolist())

E=2, K=5^2
R_rel_err: 2.996561365416923e-16
W_rel_err: 3.742895694909082e-09
pi_rel_err: 6.482387353568562e-10
v_rel_err: 2.8005804239252397e-09
dR_FISTA: -2.4424906541753444e-17
dW_FISTA: 7.742679336564251e-10
dR_kaiseki: -3.7747582837255325e-17
dW_kaiseki: 0.0
E=5, K=5^2
R_rel_err: 3.404146174359353e-11
W_rel_err: 2.4126491791406313e-08
pi_rel_err: 3.6097669949797845e-09
v_rel_err: 1.0568697620333474e-08
dR_FISTA: -3.404149406804891e-11
dW_FISTA: -1.2113967279292481e-08
dR_kaiseki: -6.661338147750939e-18
dW_kaiseki: 8.326672684688674e-17
E=8, K=5^2
R_rel_err: 7.1566424271566286e-12
W_rel_err: 7.796669581493075e-10
pi_rel_err: 1.0971801514490577e-10
v_rel_err: 3.1076621527259463e-10
dR_FISTA: -6.3694849394835275e-12
dW_FISTA: -5.734665853296405e-10
dR_kaiseki: 1.1102230246251566e-17
dW_kaiseki: 1.0658141036401502e-16
E=10, K=5^2
R_rel_err: 4.0218363348941296e-10
W_rel_err: 3.104690405370148e-08
pi_rel_err: 4.258887281662354e-09
v_rel_err: 1.114478778328211e-08
dR_FISTA: -3.57230238723

In [ ]:
E_list = [2, 5, 8, 10]
Col_list = [20, 30]
for E in E_list:
    
    for Col in Col_list:
        
        print("="*40)
        print(f'E={E}, K={Col}^2')
        
        K = Col * Col
    
        M = 0.1 * K
        N = 0.1 * K

        dF_long = pd.read_excel(fr'C:/Users/kimura/OneDrive - 国立大学法人東北大学/numerical/short_dual/log/excelfile/' \
                fr'shortlong/kaisekikai/theta=5.0/mesh/E={E}/v_proj=0.1/K={Col}^2/truevalue/jikken_F.xlsx')
        dH_long = pd.read_excel(fr'C:/Users/kimura/OneDrive - 国立大学法人東北大学/numerical/short_dual/log/excelfile/' \
                fr'shortlong/kaisekikai/theta=5.0/mesh/E={E}/v_proj=0.1/K={Col}^2/truevalue/jikken_H.xlsx')

        m_fixed = np.array(dF_long['m'].tolist())
        n_fixed = np.array(dH_long['n'].tolist()).reshape(K, K)

        # m_per = M / K
        # m_fixed = np.array([m_per] * K)
        # n_fixed = np.full((K, K), N / (K * K))
    
        
        Scaling = 3 / Col

        city_network = lattice_kai.make_lattice(Col)

        # 通勤費用パラメータ
        t = 0.1
        
        # 距離抵抗パラメータ
        tau = 0.01

        # 総土地供給量
        S_total = K
        S_bar = S_total / K

        Coordinate_Data = \
        np.array([(city_network['node_dic'][str(i)]['x_pos']*Scaling,
                   city_network['node_dic'][str(i)]['y_pos']*Scaling) for i in range(K)])
        distance_matrix = distance.squareform(distance.pdist(Coordinate_Data))

        theta_N = 5.0
        theta_M = 5.0

        # E = 5

        alpha_1 = 0.4
        alpha_2 = 0.4
        beta_1 = 0.4
        beta_2 = 0.4

        L = 0.2
        eta = 1.2
        p_proj = 0.1
        
        RW_ini = np.ones(2 * K)
        
        sci_alg = "L-BFGS-B"

        #パラメータ設定
        prm = Short_sci.Parameter(Col, distance_matrix, t, tau, Scaling, S_total, S_bar,
                    theta_M, theta_N, E, alpha_1, alpha_2, beta_1, beta_2, M, N)
        algprm = Short_sci.Algo_Parameter(L, eta, p_proj)

        dF_sci = pd.read_excel(fr'C:/Users/kimura/OneDrive - 国立大学法人東北大学/numerical/short_dual/log/excelfile/' \
            fr'short/scipy_{sci_alg}/long_shuseki/mesh/ftol=1e-08/gtol=1e-08/jac=given/E={prm.E}/K={prm.Col}^2/jikken_F.xlsx')
        dF_kaiseki = pd.read_excel(fr'C:/Users/kimura/OneDrive - 国立大学法人東北大学/numerical/short_dual/log/excelfile/' \
            fr'short/kaisekikai/long_shuseki/mesh/E={prm.E}/v_proj=None/K={prm.Col}^2/jikken_F.xlsx')
        dH_sci = pd.read_excel(fr'C:/Users/kimura/OneDrive - 国立大学法人東北大学/numerical/short_dual/log/excelfile/' \
            fr'short/scipy_{sci_alg}/long_shuseki/mesh/ftol=1e-08/gtol=1e-08/jac=given/E={prm.E}/K={prm.Col}^2/jikken_H.xlsx')
        dH_kaiseki = pd.read_excel(fr'C:/Users/kimura/OneDrive - 国立大学法人東北大学/numerical/short_dual/log/excelfile/' \
            fr'short/kaisekikai/long_shuseki/mesh/E={prm.E}/v_proj=None/K={prm.Col}^2/jikken_H.xlsx')
        
        # short = Short_sci.Short(prm, algprm, m_fixed, n_fixed)
        # dRW_ini = short.short_dual_df(RW_ini)
        # print("dRW_ini:", np.mean(dRW_ini))

        R_sci = np.array(dF_sci['R'].tolist())
        R_kaiseki = np.array(dF_kaiseki['R'].tolist())

        print("R_rel_err:", np.mean(abs((R_sci - R_kaiseki) / R_kaiseki)))
        # print("R_FISTA:", np.mean(R_FISTA))
        # print("R_kaiseki:", np.mean(R_kaiseki))

        W_sci = np.array(dF_sci['W'].tolist())
        W_kaiseki = np.array(dF_kaiseki['W'].tolist())

        print("W_rel_err:", np.mean(abs((W_sci - W_kaiseki) / W_kaiseki)))
        # print("W_FISTA:", np.mean(W_FISTA))
        # print("W_kaiseki:", np.mean(W_kaiseki))

        pi_sci = np.array(dF_sci['pi'].tolist())
        pi_kaiseki = np.array(dF_kaiseki['pi'].tolist())

        print("pi_rel_err:", np.mean(abs((pi_sci - pi_kaiseki) / pi_kaiseki)))
        # print("pi_FISTA:", np.mean(pi_FISTA))
        # print("pi_kaiseki:", np.mean(pi_kaiseki))

        v_sci = np.array(dH_sci['v'].tolist()).flatten()
        v_kaiseki = np.array(dH_kaiseki['v'].tolist()).flatten()

        v_err = []
        for i in range(len(v_sci)):
            if v_sci[i] == 0:  # ゼロ割りを防ぐための処理
                if v_kaiseki[i] == 0:
                    v_err.append(0)
                else:
                    v_err.append(float('inf'))
            else:
                v_err.append(abs((v_sci[i] - v_kaiseki[i]) / v_kaiseki[i]))
            
        # print("v_err:", v_err)
        
        print("v_rel_err:", np.mean(v_err))
        # print("v_FISTA:", v_FISTA)
        # print("v_kaiseki:", v_kaiseki)
        # print("v_FISTA_min:", np.min(v_FISTA))
        # print("v_kaiseki_min:", np.min(v_kaiseki))
        
        short = Short_sci.Short(prm, algprm, m_fixed, n_fixed)
        RW_sci = np.concatenate((R_sci, W_sci))
        dRdW_sci = short.short_dual_df(RW_sci)
        
        dR_sci = dRdW_sci[:K]
        dW_sci = dRdW_sci[K:]
        
        print("dR_sci:", np.mean(dR_sci))
        print("dW_sci:", np.mean(dW_sci))
        
        short = Short_sci.Short(prm, algprm, m_fixed, n_fixed)
        RW_kaiseki = np.concatenate((R_kaiseki, W_kaiseki))
        dRdW_kaiseki = short.short_dual_df(RW_kaiseki)
        
        dR_kaiseki = dRdW_kaiseki[:K]
        dW_kaiseki = dRdW_kaiseki[K:]
        
        print("dR_kaiseki:", np.mean(dR_kaiseki))
        print("dW_kaiseki:", np.mean(dW_kaiseki))

        #objはデータに格納されていない．
        # obj_sci = np.array(dF_sci['v'].tolist())
        # obj_FISA = np.array(dF_FISTA['v'].tolist())

In [2]:
E_list = [5, 8, 10]
Col_list = [5, 8, 10, 20, 30]
for Col in Col_list:
    
    for E in E_list:
        
        print("="*40)
        print(f'E={E}, K={Col}^2')
        
        K = Col * Col
    
        M = 0.1 * K
        N = 0.1 * K

#         dF_long = pd.read_excel(fr'C:/Users/kimura/OneDrive - 国立大学法人東北大学/numerical/short_dual/log/excelfile/' \
#                 fr'shortlong/kaisekikai/theta=5.0/mesh/E={E}/v_proj=0.1/K={Col}^2/truevalue/jikken_F.xlsx')
#         dH_long = pd.read_excel(fr'C:/Users/kimura/OneDrive - 国立大学法人東北大学/numerical/short_dual/log/excelfile/' \
#                 fr'shortlong/kaisekikai/theta=5.0/mesh/E={E}/v_proj=0.1/K={Col}^2/truevalue/jikken_H.xlsx')

#         m_fixed = np.array(dF_long['m'].tolist())
#         n_fixed = np.array(dH_long['n'].tolist()).reshape(K, K)

        m_per = M / K
        m_fixed = np.array([m_per] * K)
        n_fixed = np.full((K, K), N / (K * K))
    
        Scaling = 30 / Col
        alter_T_num = 0.5

        city_network = lattice_kai.make_lattice(Col)

        # 通勤費用パラメータ
        t = 0.1
        
        # 距離抵抗パラメータ
        tau = 0.01

        # 総土地供給量
        S_total = K
        S_bar = S_total / K

        Coordinate_Data = \
        np.array([(city_network['node_dic'][str(i)]['x_pos']*Scaling,
                   city_network['node_dic'][str(i)]['y_pos']*Scaling) for i in range(K)])
        distance_matrix = distance.squareform(distance.pdist(Coordinate_Data))

        theta_house = 5.0
        theta_firm = 5.0

        # E = 5

        alpha_1 = 0.4
        alpha_2 = 0.4
        beta_1 = 0.4
        beta_2 = 0.4

        L = 0.2
        eta = 1.2
        p_proj = 1e-3
        
        ftol = 1e-8
        gtol = 1e-8
        
        RW_ini = np.ones(2 * K)
        
        RW_proj = 1e-3
        
        sci_alg = "L-BFGS-B"

        #パラメータ設定
        prm = exp_Short_FISTA.Parameter(
                    Col, distance_matrix, t, tau, Scaling, S_total, S_bar,
                    theta_firm, theta_house, E, RW_proj, alter_T_num,
                    alpha_1, alpha_2, beta_1, beta_2, M, N)
        
        algprm = exp_Short_FISTA.Algo_Parameter(L, eta, p_proj)
        short = exp_Short_FISTA.Short(prm, algprm, m_fixed, n_fixed)

        dF_sci = pd.read_excel(fr'C:/Users/kimura/OneDrive - 国立大学法人東北大学/numerical/short_dual/exp/excelfile/' \
            fr'short/long_ichiyou/scipy_{sci_alg}/Scaling=30divCol/RW_proj={prm.RW_proj}/ftol={ftol}/gtol={gtol}/jac=given/E={prm.E}/K={prm.Col}^2/jikken_F.xlsx')
        dF_FISTA = pd.read_excel(fr'C:/Users/kimura/OneDrive - 国立大学法人東北大学/numerical/short_dual/exp/excelfile/' \
            fr'short/long_ichiyou/FISTA/Scaling=30divCol/RW_proj={prm.RW_proj}/alter_T_num={prm.alter_T_num}/L={algprm.L}/eta={algprm.eta}/p_proj={algprm.p_proj}/E={prm.E}/K={prm.Col}^2/truevalue/jikken_F.xlsx')
        dH_sci = pd.read_excel(fr'C:/Users/kimura/OneDrive - 国立大学法人東北大学/numerical/short_dual/exp/excelfile/' \
            fr'short/long_ichiyou/scipy_{sci_alg}/Scaling=30divCol/RW_proj={prm.RW_proj}/ftol={ftol}/gtol={gtol}/jac=given/E={prm.E}/K={prm.Col}^2/jikken_H.xlsx')
        dH_FISTA = pd.read_excel(fr'C:/Users/kimura/OneDrive - 国立大学法人東北大学/numerical/short_dual/exp/excelfile/' \
            fr'short/long_ichiyou/FISTA/Scaling=30divCol/RW_proj={prm.RW_proj}/alter_T_num={prm.alter_T_num}/L={algprm.L}/eta={algprm.eta}/p_proj={algprm.p_proj}/E={prm.E}/K={prm.Col}^2/truevalue/jikken_H.xlsx')
        
        # short = Short_sci.Short(prm, algprm, m_fixed, n_fixed)
        # dRW_ini = short.short_dual_df(RW_ini)
        # print("dRW_ini:", np.mean(dRW_ini))

        R_sci = np.array(dF_sci['R'].tolist())
        R_FISTA = np.array(dF_FISTA['R'].tolist())

        print("R_rel_err:", np.mean(abs((R_sci - R_FISTA) / R_sci)))
        # print("R_FISTA:", np.mean(R_FISTA))
        # print("R_kaiseki:", np.mean(R_kaiseki))

        W_sci = np.array(dF_sci['W'].tolist())
        W_FISTA = np.array(dF_FISTA['W'].tolist())

        print("W_rel_err:", np.mean(abs((W_sci - W_FISTA) / W_sci)))
        # print("W_FISTA:", np.mean(W_FISTA))
        # print("W_kaiseki:", np.mean(W_kaiseki))

        pi_sci = np.array(dF_sci['pi'].tolist())
        pi_FISTA = np.array(dF_FISTA['pi'].tolist())

        print("pi_rel_err:", np.mean(abs((pi_sci - pi_FISTA) / pi_sci)))
        # print("pi_FISTA:", np.mean(pi_FISTA))
        # print("pi_kaiseki:", np.mean(pi_kaiseki))

        v_sci = np.array(dH_sci['v'].tolist()).flatten()
        v_FISTA = np.array(dH_FISTA['v'].tolist()).flatten()

        v_err = []
        for i in range(len(v_sci)):
            if v_sci[i] == 0:  # ゼロ割りを防ぐための処理
                if v_FISTA[i] == 0:
                    v_err.append(0)
                else:
                    v_err.append(float('inf'))
            else:
                v_err.append(abs((v_sci[i] - v_FISTA[i]) / v_sci[i]))
            
        # print("v_err:", v_err)
        
        print("v_rel_err:", np.mean(v_err))
        # print("v_FISTA:", v_FISTA)
        # print("v_kaiseki:", v_kaiseki)
        # print("v_FISTA_min:", np.min(v_FISTA))
        # print("v_kaiseki_min:", np.min(v_kaiseki))
        
        short = exp_Short_sci.Short(prm, m_fixed, n_fixed)
        RW_sci = np.concatenate((R_sci, W_sci))
        dRdW_sci = short.short_dual_df(RW_sci)
        
        dR_sci = dRdW_sci[:K]
        dW_sci = dRdW_sci[K:]
        
        print("dR_sci:", np.mean(dR_sci))
        print("dW_sci:", np.mean(dW_sci))
        
        short = exp_Short_FISTA.Short(prm, algprm, m_fixed, n_fixed)
        RW_FISTA = np.concatenate((R_FISTA, W_FISTA))
        dRdW_FISTA = short.short_dual_df(RW_FISTA)
        
        dR_FISTA = dRdW_FISTA[:K]
        dW_FISTA = dRdW_FISTA[K:]
        
        print("dR_FISTA:", np.mean(dR_FISTA))
        print("dW_FISTA:", np.mean(dW_FISTA))

        #objはデータに格納されていない．
        # obj_sci = np.array(dF_sci['v'].tolist())
        # obj_FISA = np.array(dF_FISTA['v'].tolist())

E=5, K=5^2
R_rel_err: 1.5963298025430116e-05
W_rel_err: 2.332772869769621e-05
pi_rel_err: 1.9066941924773564e-06
v_rel_err: 2.154808352150833e-05
dR_sci: 1.4069495553589716e-06
dW_sci: -1.148964084136811e-06
dR_FISTA: -1.5042131459863661e-09
dW_FISTA: 3.956458801690488e-09
E=8, K=5^2
R_rel_err: 1.0946662802310347e-05
W_rel_err: 2.4712030073397582e-05
pi_rel_err: 1.9641106475702613e-06
v_rel_err: 2.2846433764373434e-05
dR_sci: -4.5913985679175425e-05
dW_sci: -4.3621786514461694e-05
dR_FISTA: -1.2615334881171947e-08
dW_FISTA: 1.7571627633494734e-08
E=10, K=5^2
R_rel_err: 2.7493422778223778e-08
W_rel_err: 2.749342273896485e-08
pi_rel_err: 5.561262346240505e-09
v_rel_err: 2.8372753583029248e-08
dR_sci: -2.6947746099015292e-08
dW_sci: -2.6947746020467012e-08
dR_FISTA: 1.3313376111545594e-08
dW_FISTA: 1.3313376127921383e-08
E=5, K=8^2
R_rel_err: 4.7739683533631305e-05
W_rel_err: 6.624559343757407e-05
pi_rel_err: 1.9984841656269516e-07
v_rel_err: 6.225975589188069e-05
dR_sci: -2.8297504233747

In [6]:
E_list = [5]
Col_list = [10]
for Col in Col_list:
    
    K = Col * Col
    
    M = 0.1 * K
    N = 0.1 * K
    
    dF_long = pd.read_excel(fr'C:/Users/kimura/OneDrive - 国立大学法人東北大学/numerical/short_dual/log/excelfile/' \
    fr'shortlong/FISTA/theta=5.0/E=10/mesh/K={Col}^2/truevalue/jikken_F.xlsx')
    dH_long = pd.read_excel(fr'C:/Users/kimura/OneDrive - 国立大学法人東北大学/numerical/short_dual/log/excelfile/' \
    fr'shortlong/FISTA/theta=5.0/E=10/mesh/K={Col}^2/truevalue/jikken_H.xlsx')

    m_fixed = np.array(dF_long['m'].tolist())
    n_fixed = np.array(dH_long['n'].tolist()).reshape(K, K)

    # m_per = M / K
    # m_fixed = np.array([m_per] * K)
    # n_fixed = np.full((K, K), N / (K * K))
    
    for E in E_list:
        
        print("="*40)
        print(f'E={E}, K={Col}^2')
        
        Scaling = 3 / Col

        city_network = lattice_kai.make_lattice(Col)

        # 通勤費用パラメータ
        t = 0.1
        
        # 距離抵抗パラメータ
        tau = 0.01

        # 総土地供給量
        S_total = K
        S_bar = S_total / K

        Coordinate_Data = \
        np.array([(city_network['node_dic'][str(i)]['x_pos']*Scaling,
                   city_network['node_dic'][str(i)]['y_pos']*Scaling) for i in range(K)])
        distance_matrix = distance.squareform(distance.pdist(Coordinate_Data))

        theta_N = 5.0
        theta_M = 5.0

        # E = 5

        alpha_1 = 0.4
        alpha_2 = 0.4
        beta_1 = 0.4
        beta_2 = 0.4

        L = 0.2
        eta = 1.2
        p_proj = 0.1

        #パラメータ設定
        prm = Short_qNewton.Parameter(Col, distance_matrix, t, tau, Scaling, S_total, S_bar,
                    theta_M, theta_N, E, alpha_1, alpha_2, beta_1, beta_2, M, N)
        algprm = Short_qNewton.Algo_Parameter(L, eta, p_proj)

        dF_qNewton = pd.read_excel(fr'C:/Users/kimura/OneDrive - 国立大学法人東北大学/numerical/short_dual/log/excelfile/' \
            fr'short/qNewton/long_shuseki/mesh/E={prm.E}/L={algprm.L}/p_proj={algprm.p_proj}/v_proj=0.1/df=None/K={prm.Col}^2/truevalue/jikken_F.xlsx')
        dF_kaiseki = pd.read_excel(fr'C:/Users/kimura/OneDrive - 国立大学法人東北大学/numerical/short_dual/log/excelfile/' \
            fr'short/FISTA/long_shuseki/mesh/E={prm.E}/kaisekikai/v_proj=0.1/K={prm.Col}^2/jikken_F.xlsx')
        dH_qNewton = pd.read_excel(fr'C:/Users/kimura/OneDrive - 国立大学法人東北大学/numerical/short_dual/log/excelfile/' \
            fr'short/qNewton/long_shuseki/mesh/E={prm.E}/L={algprm.L}/p_proj={algprm.p_proj}/v_proj=0.1/df=None/K={prm.Col}^2/truevalue/jikken_H.xlsx')
        dH_kaiseki = pd.read_excel(fr'C:/Users/kimura/OneDrive - 国立大学法人東北大学/numerical/short_dual/log/excelfile/' \
            fr'short/FISTA/long_shuseki/mesh/E={prm.E}/kaisekikai/v_proj=0.1/K={prm.Col}^2/jikken_H.xlsx')

        R_qNewton = np.array(dF_qNewton['R'].tolist())
        R_kaiseki = np.array(dF_kaiseki['R'].tolist())

        print("R_rel_err:", np.mean(abs((R_qNewton - R_kaiseki) / R_kaiseki)))
        # print("R_qNewton:", np.mean(R_qNewton))
        # print("R_kaiseki:", np.mean(R_kaiseki))

        W_qNewton = np.array(dF_qNewton['W'].tolist())
        W_kaiseki = np.array(dF_kaiseki['W'].tolist())

        print("W_rel_err:", np.mean(abs((W_qNewton - W_kaiseki) / W_kaiseki)))
        # print("W_qNewton:", np.mean(W_qNewton))
        # print("W_kaiseki:", np.mean(W_kaiseki))

        pi_qNewton = np.array(dF_qNewton['pi'].tolist())
        pi_kaiseki = np.array(dF_kaiseki['pi'].tolist())

        print("pi_rel_err:", np.mean(abs((pi_qNewton - pi_kaiseki) / pi_kaiseki)))
        # print("pi_qNewton:", np.mean(pi_qNewton))
        # print("pi_kaiseki:", np.mean(pi_kaiseki))

        v_qNewton = np.array(dH_qNewton['v'].tolist()).flatten()
        v_kaiseki = np.array(dH_kaiseki['v'].tolist()).flatten()

        v_err = []
        for i in range(len(v_qNewton)):
            if v_qNewton[i] == 0:  # ゼロ割りを防ぐための処理
                if v_kaiseki[i] == 0:
                    v_err.append(0)
                else:
                    v_err.append(float('inf'))
            else:
                v_err.append(abs((v_qNewton[i] - v_kaiseki[i]) / v_kaiseki[i]))
            
        # print("v_err:", v_err)
        
        print("v_rel_err:", np.mean(v_err))
        # print("v_qNewton:", v_qNewton)
        # print("v_kaiseki:", v_kaiseki)
        # print("v_FISTA_min:", np.min(v_FISTA))
        # print("v_kaiseki_min:", np.min(v_kaiseki))
        
        short = Short_qNewton.Short(prm, algprm, m_fixed, n_fixed)
        RW_qNewton = np.concatenate((R_qNewton, W_qNewton))
        dRdW_qNewton = short.short_dual_df(RW_qNewton)
        
        dR_qNewton = dRdW_qNewton[:K]
        dW_qNewton = dRdW_qNewton[K:]
        
        print("dR_qNewton:", np.mean(dR_qNewton))
        print("dW_qNewton:", np.mean(dW_qNewton))

        RW_kaiseki = np.concatenate((R_kaiseki, W_kaiseki))
        dRdW_kaiseki = short.short_dual_df(RW_kaiseki)
        
        dR_kaiseki = dRdW_kaiseki[:K]
        dW_kaiseki = dRdW_kaiseki[K:]
        
        print("dR_kaiseki:", np.mean(dR_kaiseki))
        print("dW_kaiseki:", np.mean(dW_kaiseki))

        #objはデータに格納されていない．
        # obj_sci = np.array(dF_sci['v'].tolist())
        # obj_FISA = np.array(dF_FISTA['v'].tolist())

E=5, K=10^2
R_rel_err: 5.132872891147867e-09
W_rel_err: 3.2862844886454863e-09
pi_rel_err: 2.871722204894149e-10
v_rel_err: 3.201998648122162e-09
dR_qNewton: 2.8883800906598367e-10
dW_qNewton: -1.3329561066033335e-11
dR_kaiseki: 2.2204460492503132e-17
dW_kaiseki: 1.5543122344752193e-17
